In [1]:
library(tidyverse)

-- Attaching packages --------------------------------------- tidyverse 1.3.2 --
v ggplot2 3.4.1     v purrr   1.0.1
v tibble  3.1.8     v dplyr   1.1.0
v tidyr   1.3.0     v stringr 1.5.0
v readr   2.1.4     v forcats 1.0.0
-- Conflicts ------------------------------------------ tidyverse_conflicts() --
x dplyr::filter() masks stats::filter()
x dplyr::lag()    masks stats::lag()


In [2]:
# Make sure all columns of a tibble are being shown
options(repr.matrix.max.rows = Inf, repr.matrix.max.cols = Inf)

In [3]:
# 1 ##########################################################################################################
# The grant data #############################################################################################

In [4]:
# Raw grant data
raw_grantdata <- read_delim("../raw/Grant.csv", delim=";", col_names=TRUE)

Rows: 84341 Columns: 24
-- Column specification --------------------------------------------------------
Delimiter: ";"
chr  (17): AllDisciplines, CallFullTitle, FundingInstrumentLevel1, FundingIn...
dbl   (4): AmountGrantedAllSets, CallDecisionYear, GrantNumber, MainDiscipli...
dttm  (3): CallEndDate, EffectiveGrantEndDate, EffectiveGrantStartDate

i Use `spec()` to retrieve the full column specification for this data.
i Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [5]:
# Keep the columns of interest
grantdata_default <- select(raw_grantdata, 
                        GrantNumber, 
                        CallDecisionYear, 
                        EffectiveGrantStartDate, 
                        EffectiveGrantEndDate, 
                        State,
                        FundingInstrumentLevel1, # Careers, projects, etc.
                        AmountGrantedAllSets,    # Approved amount
                        ResearchInstitution,     # On the level of university, university of applied science, etc.
                        InstituteCountry,
                        MainDiscipline_Level1,   # Main discipline
                        MainDiscipline_Level2,   # Subdiscipline
                        MainDiscipline           # Speciality
                    )

In [6]:
# Add the effective approved grant duration in days and years

# Duration in days
grantdata_default <- mutate(grantdata_default, GrantDurationDays=ifelse(
    as.numeric(as.Date(EffectiveGrantEndDate) - as.Date(EffectiveGrantStartDate)) < 0,
    0, # There's data where the end date is smaller than the start date
    as.numeric(as.Date(EffectiveGrantEndDate) - as.Date(EffectiveGrantStartDate))),
    .after=EffectiveGrantStartDate)

# Duration in years
grantdata_default <- mutate(grantdata_default, GrantDurationYears=GrantDurationDays/365, .after=GrantDurationDays)

In [7]:
head(grantdata_default, 3)

GrantNumber,CallDecisionYear,EffectiveGrantStartDate,GrantDurationDays,GrantDurationYears,EffectiveGrantEndDate,State,FundingInstrumentLevel1,AmountGrantedAllSets,ResearchInstitution,InstituteCountry,MainDiscipline_Level1,MainDiscipline_Level2,MainDiscipline
<dbl>,<dbl>,<dttm>,<dbl>,<dbl>,<dttm>,<chr>,<chr>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>
208399,2022,2022-10-01,608,1.665753,2024-05-31,Ongoing,Science communication,150000,University of Berne - BE,Switzerland,Humanities and Social Sciences,"Art studies, musicology, theatre and film studies, architecture",Theatre and Cinema
158034,2014,2014-12-01,1095,3.000000,2017-11-30,Completed,Science communication,11390,University of Berne - BE,Switzerland,Humanities and Social Sciences,"Art studies, musicology, theatre and film studies, architecture",Theatre and Cinema
139781,2011,2012-01-01,365,1.000000,2012-12-31,Completed,Careers,45360,Institution abroad - France,France,Humanities and Social Sciences,"Linguistics and literature, philosophy",Romance languages and literature


In [8]:
# 2 ##########################################################################################################
# The number of employess per grant ##########################################################################

In [9]:
# Raw person data
raw_persondata <- read_delim("../raw/Person.csv", delim = ";", col_names=TRUE)

Warning message:
"One or more parsing issues, call `problems()` on your data frame for details,
e.g.:
  dat <- vroom(...)
  problems(dat)"
Rows: 127057 Columns: 16
-- Column specification --------------------------------------------------------
Delimiter: ";"
chr (12): CoApplicantGrantNumber, EmployeeGrantNumber, FirstName, Gender, In...
dbl  (3): ContactPersonGrantNumber, PersonNumber, PracticePartnerGrantNumber
lgl  (1): ApplicantAbroadGrantNumber

i Use `spec()` to retrieve the full column specification for this data.
i Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [10]:
# Keep the columns of interest
persondata <- select(raw_persondata, 
                         PersonNumber, 
                         Gender,
                         EmployeeGrantNumber,
                         CoApplicantGrantNumber,
                         ProjectPartnerGrantNumber
                     )

In [11]:
head(persondata, 3)

PersonNumber,Gender,EmployeeGrantNumber,CoApplicantGrantNumber,ProjectPartnerGrantNumber
<dbl>,<chr>,<chr>,<chr>,<chr>
663397,female,156915,NA,NA
791509,male,NA,NA,216636
506604,female,135721;131339;118704;109908,NA,NA


In [12]:
# The EmployeGrantNumber contains semicolon-separated values
# For each unique value in this colunn, a new row with replicated data of the other values is generated
persondata <- separate_rows(persondata, EmployeeGrantNumber, sep = ";")

In [13]:
# Convert all the grant number columns into numeric ones so the later merge with the actual grant data will work
persondata <- mutate(persondata, 
                         EmployeeGrantNumber=ifelse(is.na(EmployeeGrantNumber), NA, as.numeric(EmployeeGrantNumber)),
                         CoApplicantGrantNumber=ifelse(is.na(CoApplicantGrantNumber), NA, as.numeric(CoApplicantGrantNumber)),
                         ProjectPartnerGrantNumber=ifelse(is.na(ProjectPartnerGrantNumber), NA, as.numeric(ProjectPartnerGrantNumber))
                     )

Warning message:
"There were 2 warnings in `mutate()`.
The first warning was:
i In argument: `CoApplicantGrantNumber = ifelse(is.na(CoApplicantGrantNumber),
  NA, as.numeric(CoApplicantGrantNumber))`.
Caused by warning in `ifelse()`:
! NAs durch Umwandlung erzeugt
i Run `dplyr::last_dplyr_warnings()` to see the 1 remaining warning."


In [14]:
# Create a dataframe for the number of employess per grant
employees_per_grant <- summarise(group_by(persondata, EmployeeGrantNumber), NrEmployees=n(), .groups="keep")
employees_per_grant <- ungroup(employees_per_grant)

In [15]:
head(employees_per_grant, 3)

EmployeeGrantNumber,NrEmployees
<dbl>,<int>
365,1
451,1
853,2


In [16]:
# Merge all the created person data into the grant data

In [17]:
# Employees per grant
grantdata_persons <- left_join(grantdata_default, employees_per_grant, by=c("GrantNumber"="EmployeeGrantNumber"))

In [18]:
# 3 ##########################################################################################################
# The number of collaborations per grant #####################################################################

In [19]:
# Raw collaboration data
raw_collaboration_data <- read_delim("../raw/OutputdataCollaboration.csv", delim = ";", col_select="GrantNumber", col_names=TRUE)

Rows: 83274 Columns: 1
-- Column specification --------------------------------------------------------
Delimiter: ";"
dbl (1): GrantNumber

i Use `spec()` to retrieve the full column specification for this data.
i Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [20]:
collaborations_per_grant <- summarise(group_by(raw_collaboration_data, GrantNumber), NrCollaborations=n(), .groups="keep")
collaborations_per_grant <- ungroup(collaborations_per_grant)

In [21]:
grantdata_collaborations <- left_join(grantdata_persons, collaborations_per_grant, by="GrantNumber")

In [22]:
# 4 ##########################################################################################################
# The number of further outputs ##############################################################################

In [24]:
# A function to read and merge output data files

# Parameters are: 
    # source:             the source file of the output data
    # sourceIdColumn:     the column of the source identifying the grant numer
    # sourceDateColumn:   the column of the source identifying the output date or year
    # newColumn:          the name of the main new column that should be added to the grant data
    # newDateColumn:      the name of the column that indicates the date of the FIRST corresponding event 
                          # (e.g. DateFirstAcademicEvent or YearFirstPublication)
    # target:             the target dataframe the output should be merged into
    # targetIdColumn:     the target column identifying the grant number

# For the given target dataframe, this function adds the total number of corresponding outputs as well as the date/year of the first output

addOutput <- function(source, sourceIdColumn, sourceDateColumn, newColumn, newDateColumn, target, targetIdColumn) {
    
    # Loading the output data
    dfx <- read_delim(source, delim = ";", col_select = c(!!sourceIdColumn, !!sourceDateColumn), col_names=TRUE)
    
    # Creating a dataframe with the total number of events as well as the oldest output date/year
    output_per_grant <- summarise(group_by(dfx, !!sym(sourceIdColumn)), !!newColumn := n(), !!newDateColumn := min(!!sym(sourceDateColumn)), .groups="keep")

    # Merging the output data into the given target
    target <- left_join(target, output_per_grant, by=sprintf("%s", targetIdColumn))
    
    return(target)
}

In [25]:
# 4 ##########################################################################################################
# Number of acedemic events per grant ########################################################################

In [26]:
grantdata_outputs_1 <- addOutput("../raw/OutputdataAcademicEvent.csv", 
                                 "GrantNumber", 
                                 "AcademicEvent_Date", 
                                 "NrAcademicEvents", 
                                 "DateFirstAcademicEvent", 
                                 grantdata_collaborations, 
                                 "GrantNumber"
                                )

Rows: 171369 Columns: 2
-- Column specification --------------------------------------------------------
Delimiter: ";"
dbl  (1): GrantNumber
dttm (1): AcademicEvent_Date

i Use `spec()` to retrieve the full column specification for this data.
i Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [27]:
# 5 ##########################################################################################################
# Number of awards per grant #################################################################################

In [28]:
grantdata_outputs_2 <- addOutput("../raw/OutputdataAward.csv",
                                 "GrantNumber", 
                                 "Award_Year",
                                 "NrAwards",
                                 "YearFirstAward",
                                 grantdata_outputs_1,
                                 "GrantNumber"
                                )

Rows: 10799 Columns: 2
-- Column specification --------------------------------------------------------
Delimiter: ";"
dbl (2): Award_Year, GrantNumber

i Use `spec()` to retrieve the full column specification for this data.
i Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [29]:
# 6 ##########################################################################################################
# Number of datasets per grant ###############################################################################

In [30]:
grantdata_outputs_3 <- addOutput("../raw/OutputdataDataSet.csv",
                                 "GrantNumber", 
                                 "DataSet_PublicationDate",
                                 "NrDatasets",
                                 "DateFirstDataset",
                                 grantdata_outputs_2,
                                 "GrantNumber"
                                )

Rows: 4264 Columns: 2
-- Column specification --------------------------------------------------------
Delimiter: ";"
dbl  (1): GrantNumber
dttm (1): DataSet_PublicationDate

i Use `spec()` to retrieve the full column specification for this data.
i Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [31]:
# 7 ##########################################################################################################
# Number of knowledge transfer events per grant ##############################################################

In [32]:
grantdata_outputs_4 <- addOutput("../raw/OutputdataKnowledgeTransferEvent.csv",
                                 "GrantNumber", 
                                 "KnowledgeTransferEvent_Date",
                                 "NrKnowledgeTransfer",
                                 "DateFirstKnowledgeTransfer",
                                 grantdata_outputs_3,
                                 "GrantNumber"
                                )

Rows: 16374 Columns: 2
-- Column specification --------------------------------------------------------
Delimiter: ";"
dbl  (1): GrantNumber
dttm (1): KnowledgeTransferEvent_Date

i Use `spec()` to retrieve the full column specification for this data.
i Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [33]:
# 8 ##########################################################################################################
# Number of public communications per grant ##################################################################

In [34]:
grantdata_outputs_5 <- addOutput("../raw/OutputdataPublicCommunication.csv",
                                 "GrantNumber", 
                                 "PublicCommunication_Year",
                                 "NrCommunication",
                                 "YearFirstCommunication",
                                 grantdata_outputs_4,
                                 "GrantNumber"
                                )

Rows: 39365 Columns: 2
-- Column specification --------------------------------------------------------
Delimiter: ";"
dbl (2): GrantNumber, PublicCommunication_Year

i Use `spec()` to retrieve the full column specification for this data.
i Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [35]:
# 9 ##########################################################################################################
# Number of publications per grant ###########################################################################

In [2]:
# The original was too big for GitHub, so it was split. The two files are combined again first.
data_1 <- read_delim("../raw/OutputdataScientificPublication-1.csv", delim = ";", col_names=TRUE)
data_2 <- read_delim("../raw/OutputdataScientificPublication-2.csv", delim = ";", col_names=TRUE)

# Combine the two dataframes into one
combined_data <- bind_rows(data_1, data_2)

# Save the combined dataframe as a new CSV file
write.table(combined_data, "../raw/OutputdataScientificPublication.csv", sep=";", fileEncoding="UTF-8", row.names = FALSE)

Rows: 84730 Columns: 23
-- Column specification --------------------------------------------------------
Delimiter: ";"
chr (20): ScientificPublication_Abstract, ScientificPublication_Author, Scie...
dbl  (3): GrantNumber, ScientificPublication_OpenAccessStatusYesNo, Scientif...

i Use `spec()` to retrieve the full column specification for this data.
i Specify the column types or set `show_col_types = FALSE` to quiet this message.
Rows: 88482 Columns: 23
-- Column specification --------------------------------------------------------
Delimiter: ";"
chr (20): ScientificPublication_Abstract, ScientificPublication_Author, Scie...
dbl  (3): GrantNumber, ScientificPublication_OpenAccessStatusYesNo, Scientif...

i Use `spec()` to retrieve the full column specification for this data.
i Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [36]:
grantdata_outputs_6 <- addOutput("../raw/OutputdataScientificPublication.csv",
                                 "GrantNumber", 
                                 "ScientificPublication_Year",
                                 "NrTotalPublications",
                                 "YearFirstPublication",
                                 grantdata_outputs_5,
                                 "GrantNumber"
                                )

Rows: 173212 Columns: 2
-- Column specification --------------------------------------------------------
Delimiter: ";"
dbl (2): GrantNumber, ScientificPublication_Year

i Use `spec()` to retrieve the full column specification for this data.
i Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [37]:
# 10 #########################################################################################################
# Number of practical applications per grant #################################################################

In [38]:
grantdata_outputs_7 <- addOutput("../raw/OutputdataUseInspired.csv",
                                 "GrantNumber", 
                                 "UseInspired_Year",
                                 "NrUses",
                                 "YearFirstUse",
                                 grantdata_outputs_6,
                                 "GrantNumber"
                                )

Rows: 4650 Columns: 2
-- Column specification --------------------------------------------------------
Delimiter: ";"
dbl (2): GrantNumber, UseInspired_Year

i Use `spec()` to retrieve the full column specification for this data.
i Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [39]:
# NA --> 0 in certain columns
columns_to_mutate <- c(
    "NrEmployees", 
    "NrCollaborations",
    "NrAcademicEvents",
    "NrAwards",
    "NrDatasets",
    "NrKnowledgeTransfer",
    "NrCommunication",
    "NrUses",
    "NrTotalPublications"
)

grantdata_outputs <- mutate(grantdata_outputs_7, across(all_of(columns_to_mutate), ~ ifelse(is.na(.), 0, .)))

In [40]:
# Add the total number of outputs (NrCollaborations is not considered an output !!!)
grantdata_outputs <- mutate(grantdata_outputs, NrTotalOutputs = NrAcademicEvents + 
                                NrAwards +
                                NrDatasets + 
                                NrKnowledgeTransfer + 
                                NrCommunication + 
                                NrTotalPublications + 
                                NrUses
                           )

In [41]:
# NA --> 0
grantdata_almost <- mutate(grantdata_outputs, NrTotalOutputs = ifelse(is.na(NrTotalOutputs), 0, NrTotalOutputs))

In [42]:
dim(grantdata_almost)
head(grantdata_almost, 3)
summary(grantdata_almost)
str(grantdata_almost)

[1] 84341    31

GrantNumber,CallDecisionYear,EffectiveGrantStartDate,GrantDurationDays,GrantDurationYears,EffectiveGrantEndDate,State,FundingInstrumentLevel1,AmountGrantedAllSets,ResearchInstitution,InstituteCountry,MainDiscipline_Level1,MainDiscipline_Level2,MainDiscipline,NrEmployees,NrCollaborations,NrAcademicEvents,DateFirstAcademicEvent,NrAwards,YearFirstAward,NrDatasets,DateFirstDataset,NrKnowledgeTransfer,DateFirstKnowledgeTransfer,NrCommunication,YearFirstCommunication,NrTotalPublications,YearFirstPublication,NrUses,YearFirstUse,NrTotalOutputs
<dbl>,<dbl>,<dttm>,<dbl>,<dbl>,<dttm>,<chr>,<chr>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dttm>,<dbl>,<dbl>,<dbl>,<dttm>,<dbl>,<dttm>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
208399,2022,2022-10-01,608,1.665753,2024-05-31,Ongoing,Science communication,150000,University of Berne - BE,Switzerland,Humanities and Social Sciences,"Art studies, musicology, theatre and film studies, architecture",Theatre and Cinema,0,0,0,NA,0,NA,0,NA,0,NA,0,NA,0,NA,0,NA,0
158034,2014,2014-12-01,1095,3.000000,2017-11-30,Completed,Science communication,11390,University of Berne - BE,Switzerland,Humanities and Social Sciences,"Art studies, musicology, theatre and film studies, architecture",Theatre and Cinema,0,1,1,2017-09-27,0,NA,0,NA,0,NA,0,NA,1,NA,0,NA,2
139781,2011,2012-01-01,365,1.000000,2012-12-31,Completed,Careers,45360,Institution abroad - France,France,Humanities and Social Sciences,"Linguistics and literature, philosophy",Romance languages and literature,0,0,0,NA,0,NA,0,NA,0,NA,0,NA,0,NA,0,NA,0


  GrantNumber     CallDecisionYear EffectiveGrantStartDate      
 Min.   :     1   Min.   :1975     Min.   :1975-06-01 00:00:00  
 1st Qu.: 47261   1st Qu.:1996     1st Qu.:1996-11-01 00:00:00  
 Median :121472   Median :2008     Median :2008-10-01 00:00:00  
 Mean   :110628   Mean   :2005     Mean   :2006-02-17 11:43:46  
 3rd Qu.:165376   3rd Qu.:2015     3rd Qu.:2016-06-01 00:00:00  
 Max.   :220283   Max.   :2023     Max.   :2024-07-01 00:00:00  
                                                                
 GrantDurationDays GrantDurationYears EffectiveGrantEndDate        
 Min.   :   0.0    Min.   : 0.0000    Min.   :1975-12-31 00:00:00  
 1st Qu.: 364.0    1st Qu.: 0.9973    1st Qu.:1999-02-28 00:00:00  
 Median : 729.0    Median : 1.9973    Median :2010-08-31 00:00:00  
 Mean   : 750.9    Mean   : 2.0573    Mean   :2008-03-09 10:02:50  
 3rd Qu.:1095.0    3rd Qu.: 3.0000    3rd Qu.:2018-07-31 00:00:00  
 Max.   :5477.0    Max.   :15.0055    Max.   :2029-06-30 00:00:00  
    

tibble [84,341 x 31] (S3: tbl_df/tbl/data.frame)
 $ GrantNumber               : num [1:84341] 208399 158034 139781 179533 140815 ...
 $ CallDecisionYear          : num [1:84341] 2022 2014 2011 2017 2012 ...
 $ EffectiveGrantStartDate   : POSIXct[1:84341], format: "2022-10-01" "2014-12-01" ...
 $ GrantDurationDays         : num [1:84341] 608 1095 365 27 1460 ...
 $ GrantDurationYears        : num [1:84341] 1.666 3 1 0.074 4 ...
 $ EffectiveGrantEndDate     : POSIXct[1:84341], format: "2024-05-31" "2017-11-30" ...
 $ State                     : chr [1:84341] "Ongoing" "Completed" "Completed" "Completed" ...
 $ FundingInstrumentLevel1   : chr [1:84341] "Science communication" "Science communication" "Careers" "Science communication" ...
 $ AmountGrantedAllSets      : num [1:84341] 150000 11390 45360 8000 243562 ...
 $ ResearchInstitution       : chr [1:84341] "University of Berne - BE" "University of Berne - BE" "Institution abroad - France" "University of Berne - BE" ...
 $ InstituteCoun

In [43]:
# Add number of outputs per year

# Academic events
grantdata_outputs_per_year <- mutate(grantdata_almost, NrAcademicEventsPerYear=NrAcademicEvents/GrantDurationYears, .after=NrAcademicEvents)

# Awards
grantdata_outputs_per_year <- mutate(grantdata_outputs_per_year, NrAwardsPerYear=NrAwards/GrantDurationYears, .after=NrAwards)

# Datasets
grantdata_outputs_per_year <- mutate(grantdata_outputs_per_year, NrDatasetsPerYear=NrDatasets/GrantDurationYears, .after=NrDatasets)

# Knowledge transfer
grantdata_outputs_per_year <- mutate(grantdata_outputs_per_year, NrKnowledgeTransferPerYear=NrKnowledgeTransfer/GrantDurationYears, .after=NrKnowledgeTransfer)

# Communications
grantdata_outputs_per_year <- mutate(grantdata_outputs_per_year, NrCommunicationPerYear=NrCommunication/GrantDurationYears, .after=NrCommunication)

# Publications
grantdata_outputs_per_year <- mutate(grantdata_outputs_per_year, NrTotalPublicationsPerYear=NrTotalPublications/GrantDurationYears, .after=NrTotalPublications)

# Uses
grantdata_outputs_per_year <- mutate(grantdata_outputs_per_year, NrUsesPerYear=NrUses/GrantDurationYears, .after=NrUses)

# Total outputs
grantdata_outputs_per_year <- mutate(grantdata_outputs_per_year, NrTotalOutputsPerYear=NrTotalOutputs/GrantDurationYears, .after=NrTotalOutputs)

In [44]:
# Add more numbers per year

# Amount granted per year
grantdata <- mutate(grantdata_outputs_per_year, AmountPerYear=ifelse(GrantDurationYears == 0, 0, AmountGrantedAllSets/GrantDurationYears), .after=AmountGrantedAllSets)

# Amount granted per employee
grantdata <- mutate(grantdata, AmountPerEmployee=ifelse(NrEmployees == 0, 0, AmountGrantedAllSets/NrEmployees), .after=AmountPerYear)
 
# Amount granted per employee and year
grantdata <- mutate(grantdata, AmountPerEmployeeAndYear=ifelse(AmountPerEmployee == 0, 0, AmountPerEmployee/GrantDurationYears), .after=AmountPerEmployee)

# Number of collaborations per year
grantdata <- mutate(grantdata, NrCollaborationsPerYear=NrCollaborations/GrantDurationYears, .after=NrCollaborations)

# Number of employees per year
grantdata <- mutate(grantdata, NrEmployeesPerYear=ifelse(NrEmployees == 0, 0, NrEmployees/GrantDurationYears), .after=NrEmployees)

In [45]:
head(grantdata)

GrantNumber,CallDecisionYear,EffectiveGrantStartDate,GrantDurationDays,GrantDurationYears,EffectiveGrantEndDate,State,FundingInstrumentLevel1,AmountGrantedAllSets,AmountPerYear,AmountPerEmployee,AmountPerEmployeeAndYear,ResearchInstitution,InstituteCountry,MainDiscipline_Level1,MainDiscipline_Level2,MainDiscipline,NrEmployees,NrEmployeesPerYear,NrCollaborations,NrCollaborationsPerYear,NrAcademicEvents,NrAcademicEventsPerYear,DateFirstAcademicEvent,NrAwards,NrAwardsPerYear,YearFirstAward,NrDatasets,NrDatasetsPerYear,DateFirstDataset,NrKnowledgeTransfer,NrKnowledgeTransferPerYear,DateFirstKnowledgeTransfer,NrCommunication,NrCommunicationPerYear,YearFirstCommunication,NrTotalPublications,NrTotalPublicationsPerYear,YearFirstPublication,NrUses,NrUsesPerYear,YearFirstUse,NrTotalOutputs,NrTotalOutputsPerYear
<dbl>,<dbl>,<dttm>,<dbl>,<dbl>,<dttm>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dttm>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dttm>,<dbl>,<dbl>,<dttm>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
208399,2022,2022-10-01,608,1.6657534,2024-05-31,Ongoing,Science communication,150000,90049.342,0,0.00,University of Berne - BE,Switzerland,Humanities and Social Sciences,"Art studies, musicology, theatre and film studies, architecture",Theatre and Cinema,0,0.000000,0,0.0000000,0,0.0000000,NA,0,0.00,NA,0,0,NA,0,0,NA,0,0,NA,0,0.0000000,NA,0,0,NA,0,0.0000000
158034,2014,2014-12-01,1095,3.0000000,2017-11-30,Completed,Science communication,11390,3796.667,0,0.00,University of Berne - BE,Switzerland,Humanities and Social Sciences,"Art studies, musicology, theatre and film studies, architecture",Theatre and Cinema,0,0.000000,1,0.3333333,1,0.3333333,2017-09-27,0,0.00,NA,0,0,NA,0,0,NA,0,0,NA,1,0.3333333,NA,0,0,NA,2,0.6666667
139781,2011,2012-01-01,365,1.0000000,2012-12-31,Completed,Careers,45360,45360.000,0,0.00,Institution abroad - France,France,Humanities and Social Sciences,"Linguistics and literature, philosophy",Romance languages and literature,0,0.000000,0,0.0000000,0,0.0000000,NA,0,0.00,NA,0,0,NA,0,0,NA,0,0,NA,0,0.0000000,NA,0,0,NA,0,0.0000000
179533,2017,2018-02-01,27,0.0739726,2018-02-28,Completed,Science communication,8000,108148.148,0,0.00,University of Berne - BE,Switzerland,Humanities and Social Sciences,"Theology & religious studies, history, classical studies, archaeology, prehistory and early history",General history (without pre-and early history),0,0.000000,0,0.0000000,0,0.0000000,NA,0,0.00,NA,0,0,NA,0,0,NA,0,0,NA,1,13.5185185,NA,0,0,NA,1,13.5185185
140815,2012,2012-09-01,1460,4.0000000,2016-08-31,Completed,Projects,243562,60890.500,243562,60890.50,Swiss Federal Institute of Aquatic Science and Technology - EAWAG,Switzerland,"Mathematics, Natural- and Engineering Sciences",Environmental Sciences,Other disciplines of Environmental Sciences,1,0.250000,1,0.2500000,5,1.2500000,2013-08-25,1,0.25,2015,0,0,NA,0,0,NA,0,0,NA,3,0.7500000,2016,0,0,NA,9,2.2500000
134801,2011,2011-04-01,305,0.8356164,2012-01-31,Completed,Projects,51228,61305.639,51228,61305.64,Swiss Federal Institute of Aquatic Science and Technology - EAWAG,Switzerland,"Mathematics, Natural- and Engineering Sciences",Environmental Sciences,Other disciplines of Environmental Sciences,1,1.196721,0,0.0000000,1,1.1967213,2011-08-28,0,0.00,NA,0,0,NA,0,0,NA,0,0,NA,2,2.3934426,2013,0,0,NA,3,3.5901639


In [46]:
# Write enriched grant data to file
write.table(grantdata, "1_GrantsWithOutputs.csv", sep=";", fileEncoding="UTF-8", row.names = FALSE)